In [4]:
import boto3
import pandas as pd

In [7]:


# create an S3 client object
s3 = boto3.client('s3')

# specify the name of your bucket and file
bucket_name = 'capstonehaystacks'
file_name = 'GA_LISTINGS_SALES_V2.csv'

# download the file from S3 to your local environment
s3.download_file(bucket_name, file_name, file_name)

# read the CSV file using pandas

df = pd.read_csv(file_name, index_col=False)

# view the contents of the CSV file
print(df.head())


   Unnamed: 0   latitude  longitude       full_street_address       city  \
0           0  30.725605 -83.671772  0 Johnson Road #10026497    Quitman   
1           1  30.781796 -83.558475          505 S Lee Street    Quitman   
2           2  30.864096 -81.622765      33 River Oaks Street   Woodbine   
3           3  30.796300 -81.681675     605 E Lawnwood Avenue  Kingsland   
4           4  30.762972 -81.660240    84 Whippoorwill Circle  Kingsland   

     county_name  beds  baths_full  baths_half  square_footage  ...  \
0  Brooks-County   NaN         NaN         NaN             NaN  ...   
1  Brooks-County   3.0         1.0         NaN          1460.0  ...   
2  Camden-County   3.0         2.0         NaN          1254.0  ...   
3  Camden-County   2.0         2.0         NaN          1016.0  ...   
4  Camden-County   3.0         2.0         NaN          1618.0  ...   

                                   details  special_features unit_count  \
0                    Lots/Land, 4.93 Acre

In [8]:
df

,Unnamed: 0,latitude,longitude,full_street_address,city,county_name,beds,baths_full,baths_half,square_footage,...,details,special_features,unit_count,price,transaction_type,listing_status,listing_special_features,census_state_name,census_county_name,zip
0,0,30.725605,-83.671772,0 Johnson Road #10026497,Quitman,Brooks-County,NaN,NaN,NaN,NaN,...,"Lots/Land, 4.93 Acres",0,NaN,51900,1,0,0,Georgia,"Brooks, GA",31643
1,1,30.781796,-83.558475,505 S Lee Street,Quitman,Brooks-County,3.0,1.0,NaN,1460.0,...,"Detached, 3 Beds, 1 Bath, 1,460 Sq Ft",0,NaN,99000,1,1,0,Georgia,"Brooks, GA",31643
2,2,30.864096,-81.622765,33 River Oaks Street,Woodbine,Camden-County,3.0,2.0,NaN,1254.0,...,"Detached, 3 Beds, 2 Baths, 1,254 Sq Ft",0,NaN,245816,1,1,0,Georgia,"Camden, GA",31569
3,3,30.796300,-81.681675,605 E Lawnwood Avenue,Kingsland,Camden-County,2.0,2.0,NaN,1016.0,...,"Townhouse, 2 Beds, 2 Baths, 1,016 Sq Ft",0,NaN,155000,1,1,0,Georgia,"Camden, GA",31548
4,4,30.762972,-81.660240,84 Whippoorwill Circle,Kingsland,Camden-County,3.0,2.0,NaN,1618.0,...,"Detached, 3 Beds, 2 Baths, 1,618 Sq Ft",0,NaN,200000,1,1,0,Georgia,"Camden, GA",31548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31059,31059,34.570284,-83.261971,6 Deer Chase #LOT 6,Toccoa,Stephens-County,NaN,NaN,NaN,NaN,...,"Lots/Land, 1.03 Acres",0,NaN,24900,1,1,0,Georgia,"Stephens, GA",30577
31060,31060,34.564495,-83.375946,LOT 1B Trestle Ridge Road,Toccoa,Stephens-County,NaN,NaN,NaN,NaN,...,"Lots/Land, 3.42 Acres",0,NaN,37750,1,1,0,Georgia,"Stephens, GA",30577
31061,31061,34.551258,-83.281332,3877 Georgia 17,Eastanollee,Stephens-County,NaN,NaN,NaN,NaN,...,"Other, 2.23 Acres",0,NaN,850000,1,1,0,Georgia,"Stephens, GA",30538
31062,31062,34.548427,-83.278913,3877 Georgia 17,Toccoa,Stephens-County,NaN,NaN,NaN,NaN,...,"Commercial, 2.23 Acres",2,NaN,850000,1,1,2,Georgia,"Stephens, GA",30577


In [9]:
df = df.drop('Unnamed: 0', axis=1)


In [10]:
df

,latitude,longitude,full_street_address,city,county_name,beds,baths_full,baths_half,square_footage,lot_size,...,details,special_features,unit_count,price,transaction_type,listing_status,listing_special_features,census_state_name,census_county_name,zip
0,30.725605,-83.671772,0 Johnson Road #10026497,Quitman,Brooks-County,NaN,NaN,NaN,NaN,4.0,...,"Lots/Land, 4.93 Acres",0,NaN,51900,1,0,0,Georgia,"Brooks, GA",31643
1,30.781796,-83.558475,505 S Lee Street,Quitman,Brooks-County,3.0,1.0,NaN,1460.0,0.0,...,"Detached, 3 Beds, 1 Bath, 1,460 Sq Ft",0,NaN,99000,1,1,0,Georgia,"Brooks, GA",31643
2,30.864096,-81.622765,33 River Oaks Street,Woodbine,Camden-County,3.0,2.0,NaN,1254.0,0.0,...,"Detached, 3 Beds, 2 Baths, 1,254 Sq Ft",0,NaN,245816,1,1,0,Georgia,"Camden, GA",31569
3,30.796300,-81.681675,605 E Lawnwood Avenue,Kingsland,Camden-County,2.0,2.0,NaN,1016.0,0.0,...,"Townhouse, 2 Beds, 2 Baths, 1,016 Sq Ft",0,NaN,155000,1,1,0,Georgia,"Camden, GA",31548
4,30.762972,-81.660240,84 Whippoorwill Circle,Kingsland,Camden-County,3.0,2.0,NaN,1618.0,0.0,...,"Detached, 3 Beds, 2 Baths, 1,618 Sq Ft",0,NaN,200000,1,1,0,Georgia,"Camden, GA",31548
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31059,34.570284,-83.261971,6 Deer Chase #LOT 6,Toccoa,Stephens-County,NaN,NaN,NaN,NaN,1.0,...,"Lots/Land, 1.03 Acres",0,NaN,24900,1,1,0,Georgia,"Stephens, GA",30577
31060,34.564495,-83.375946,LOT 1B Trestle Ridge Road,Toccoa,Stephens-County,NaN,NaN,NaN,NaN,3.0,...,"Lots/Land, 3.42 Acres",0,NaN,37750,1,1,0,Georgia,"Stephens, GA",30577
31061,34.551258,-83.281332,3877 Georgia 17,Eastanollee,Stephens-County,NaN,NaN,NaN,NaN,2.0,...,"Other, 2.23 Acres",0,NaN,850000,1,1,0,Georgia,"Stephens, GA",30538
31062,34.548427,-83.278913,3877 Georgia 17,Toccoa,Stephens-County,NaN,NaN,NaN,NaN,2.0,...,"Commercial, 2.23 Acres",2,NaN,850000,1,1,2,Georgia,"Stephens, GA",30577
